In [1]:
%load_ext autoreload
%autoreload

import io
import os
import sys
import django
import glob
import json
import re
import requests
from IPython.core.debugger import set_trace
from pathlib import Path
from django.core.files.base import ContentFile
from rest_framework import serializers
from tqdm.notebook import tqdm
from google.cloud import vision
from google.cloud import storage
from PIL import Image, ImageSequence
from gensim.models import Doc2Vec
from IPython.core.debugger import set_trace
import IPython
import collections
from googletrans import Translator
from google.cloud import translate_v2 as translate

parentdir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parentdir)
django.setup()

import getch.models as m
from django.db.models import F, Q, Sum, Count, Case, When, IntegerField, Value
from django.apps import apps
from django_currentuser.middleware import get_current_user
from datetime import datetime, timedelta
import time
import pandas as pd

# 한글태그 필드 채우기

In [10]:
get_ko = lambda x: ' '.join(re.compile(r'[\u3131-\u3163\uac00-\ud7a3]+|[ㄱ-ㅣ]+').findall(x))

for _pix in tqdm(m.Pix.objects.order_by('-id')[:]):
    _tokens = _pix.tokens
    _pix.tokens_ko = get_ko(_tokens)
    _pix.save()

  0%|          | 0/4298 [00:00<?, ?it/s]

# 선택한(pos) & 선택하지않은(neg) 토큰 쿼리하기

In [2]:
# tokens = m.BalancegameRecord.objects.filter(who__user__email='sohee991110@gmail.com').select_related('pix_0', 'pix_1', 'pix_0__tokens_ko', 'pix_1__tokens_ko').annotate(
tokens = m.BalancegameRecord.objects.filter(who__user__email='sohee991110@gmail.com').annotate(    
    pos_tokens=Case(
        When(chosen=0, then=F('pix_0__tokens_ko')),
        When(chosen=1, then=F('pix_1__tokens_ko')),
        default=Value(''),
    ),
    neg_tokens=Case(
        When(chosen=0, then=F('pix_1__tokens_ko')),
        When(chosen=1, then=F('pix_0__tokens_ko')),
        default=Value(''),
    ),
)

In [3]:
pos_tokens = tokens.values_list('pos_tokens', flat=True)
neg_tokens = tokens.values_list('neg_tokens', flat=True)

In [4]:
pos_stat = collections.Counter(' '.join(pos_tokens).split(' ')); pos_stat
neg_stat = collections.Counter(' '.join(neg_tokens).split(' ')); neg_stat;

In [5]:
(pos_stat - neg_stat).most_common(10)

[('자켓', 17),
 ('티셔츠', 14),
 ('휴양', 13),
 ('몸짓', 12),
 ('가방', 11),
 ('청', 11),
 ('야구모자', 10),
 ('반바지', 9),
 ('복부', 9),
 ('캡', 9)]

In [53]:
token_list = []

for _pix in tqdm(m.Pix.objects.all()[:]):
    token_list += _pix.tokens_ko.split()

  0%|          | 0/4298 [00:00<?, ?it/s]

In [54]:
len(token_list)

134735

In [58]:
collections.Counter(token_list).most_common(500)

[('소매', 3812),
 ('스트리트패션', 3531),
 ('패션디자인', 3264),
 ('티셔츠', 3091),
 ('허리', 2794),
 ('패션액세서리', 2730),
 ('강청색', 2503),
 ('무릎', 2277),
 ('블레이저', 2267),
 ('칼라', 2228),
 ('청', 2219),
 ('격식있는옷차림', 2136),
 ('운동복', 2019),
 ('안경', 1996),
 ('인간의다리', 1928),
 ('무늬', 1924),
 ('목', 1896),
 ('대퇴골', 1792),
 ('제복', 1569),
 ('패션모델', 1542),
 ('드레스셔츠', 1391),
 ('포켓', 1389),
 ('회색', 1315),
 ('모피', 1308),
 ('어깨', 1278),
 ('몸짓', 1249),
 ('겉옷', 1218),
 ('행사', 1169),
 ('가방', 1156),
 ('팔꿈치', 1078),
 ('좌석', 1020),
 ('자켓', 1013),
 ('단추', 997),
 ('다리', 991),
 ('마젠타', 856),
 ('구두', 843),
 ('인간의몸', 833),
 ('모자', 803),
 ('폰트', 778),
 ('팔', 775),
 ('셔츠', 775),
 ('캡', 766),
 ('상단', 737),
 ('트렁크', 737),
 ('후드', 717),
 ('관절', 684),
 ('스니커즈', 679),
 ('머리', 653),
 ('저지', 640),
 ('복부', 635),
 ('보석류', 634),
 ('수하물및가방', 630),
 ('바닥', 626),
 ('긴머리', 620),
 ('사무직노동자', 611),
 ('벨트', 597),
 ('서있는', 590),
 ('가죽', 571),
 ('운동화', 565),
 ('복숭아', 548),
 ('코트', 542),
 ('플래시사진', 508),
 ('색안경', 507),
 ('맨투맨', 493),
 ('재킷', 478),
 ('격자무늬

In [12]:
df = pd.read_excel('image tags alternatives.xlsx', index_col=[0], usecols=[2,3,4,5], header=[1]); df

,제거,유지,변경
태그,,,
소매,NaN,NaN,티셔츠
스트리트패션,NaN,NaN,스트리트
패션디자인,x,NaN,NaN
티셔츠,NaN,o,NaN
허리,x,NaN,NaN
...,...,...,...
모던,NaN,o,NaN
바지수트,NaN,o,NaN
모디파이드,NaN,o,NaN


In [28]:
df2 = df[df['유지'].isnull()].dropna(axis='columns', how='all'); df2

,제거,변경
태그,,
소매,NaN,티셔츠
스트리트패션,NaN,스트리트
패션디자인,x,NaN
허리,x,NaN
패션액세서리,NaN,액세서리
...,...,...
속옷,NaN,언더웨어
악기,NaN,음악
퀄리티,x,NaN


In [44]:
toks = m.Pix.objects.last().tokens_ko; toks

'의류 청바지 물 하얀 우산 하늘 위로 허리 대퇴골 여가 야외가구 여행 무릎 흑발 트렁크 장난 바닷가 아름다움 청 인간의다리 긴머리 복부 팔꿈치 갈색머리 경치 좌석 티셔츠 안경 패션액세서리 슬리브리스 나시 골지 데님 팬츠'

In [46]:
def token_modify(toks):
    _toks_new = []
    _toks = toks.split()

    for _tok in _toks:
        if _tok in df2.index:
            row = df2.loc[_tok]

            if not pd.isna(row['제거']):
                pass

            elif not pd.isna(row['변경']):
                _toks_new.append(row['변경'])

        else:
            _toks_new.append(_tok)

    return ' '.join(_toks_new)

In [47]:
token_modify(toks)

'청바지 바다 화이트 우산 편한복장 야외가구 여행룩 호캉스 데님 바지 티셔츠 안경 액세서리 슬리브리스 나시 골지 데님 팬츠'

In [53]:
for _pix in tqdm(m.Pix.objects.order_by('-id')[:]):
    _pix.tokens_ko = token_modify(_pix.tokens_ko)
    _pix.save()

  0%|          | 0/4298 [00:00<?, ?it/s]